In [1]:
from functions_questions import add_to_file
from functions_variable_overview import *

## Create Table 

Read in description table 

In [18]:
table = pd.read_csv("tables/covid_variable_description.csv", sep=";")

Rename columns so they are easier to handle for the functions

In [19]:
table = table.rename(
    columns={
        "L_CoronavirusImpact_wave3_4p.dta": "wave#3",
        "L_CoronavirusImpact_4.0p.dta": "wave#2",
        "L_CoronavirusImpact_3p.dta": "wave#1",
        "L_CoronavirusImpact_wave4_4p.dta": "wave#4",
        "L_CoronavirusImpact_wave5_3p.dta": "wave#5",
        "new_name": "variable",
        "topic_english": "Topic",
        "type": "Type",
        "new_name": "Variable",
    }
)

In [20]:
table.head()

,Variable,reference period other than survey period,label_english,wave#5,wave#4,wave#3,wave#2,wave#1,Type,categories_english,ordered,Topic,question_group,routing_wave_5
0,l_2m_infected,NaN,How likely do you think it is that you will be...,NaN,NaN,NaN,NaN,q1a,category,not at all likely|very unlikely|rather unlikel...,True,Health Beliefs,q1,NaN
1,l_2m_acquaintance_infected,NaN,How likely do you think it is that someone in ...,NaN,NaN,NaN,NaN,q1b,category,not at all likely|very unlikely|rather unlikel...,True,Health Beliefs,q1,NaN
2,l_2m_hospital_if_infect_self,NaN,How likely do you think it is that you will ne...,NaN,NaN,NaN,NaN,q1c,category,not at all likely|very unlikely|rather unlikel...,True,Health Beliefs,q1,NaN
3,l_2m_quarantine,NaN,How likely do you think it is that you will be...,NaN,NaN,NaN,NaN,q1d,category,not at all likely|very unlikely|rather unlikel...,True,Health Beliefs,q1,NaN
4,l_2m_infected_and_pass_on,NaN,How likely do you think it is that you will ge...,NaN,NaN,NaN,NaN,q1e,category,not at all likely|very unlikely|rather unlikel...,True,Health Beliefs,q1,NaN


Select wave and language indicator. 
- `language` should be one lower-case letter. 
- `waves` should be a list of one or more values.

In [21]:
waves = [1,2,3,4,5]
language_short = "e"
language_long = "english"

Create decription table and save to csv. If the table only includes one wave, it is saved to the respective language folder. If it is created for multiple waves, it is saved to "/docs/source/".

In [22]:
if len(waves) == 1:
    variable_table = create_description_table(table, waves, language_short)
    path = f"../docs/source/wave-{waves[0]}/"
else:
    variable_table = create_description_table(
        table, waves, language_short, routing=False
    )
    # Update path for multiple-wave overview:
    path = f"../docs/source/"

In [23]:
variable_table

,Variable,Type,Topic,Reference Period Other Than Survey Period,Links Wave 1,Links Wave 2,Links Wave 3,Links Wave 4,Links Wave 5
0,l_2m_infected,Ordered Categorical,Health Beliefs,,:ref:`🔗 <w1e-q1>`,,,,
1,l_2m_acquaintance_infected,Ordered Categorical,Health Beliefs,,:ref:`🔗 <w1e-q1>`,,,,
2,l_2m_hospital_if_infect_self,Ordered Categorical,Health Beliefs,,:ref:`🔗 <w1e-q1>`,,,,
3,l_2m_quarantine,Ordered Categorical,Health Beliefs,,:ref:`🔗 <w1e-q1>`,,,,
4,l_2m_infected_and_pass_on,Ordered Categorical,Health Beliefs,,:ref:`🔗 <w1e-q1>`,,,,
...,...,...,...,...,...,...,...,...,...
1336,seeking_job,bool,Job Search,,,,,,:ref:`🔗 <w5e-JobSearch>`
1337,seeking_diff_profession,bool,Job Search,,,,,,:ref:`🔗 <w5e-DifferentJobs>`
1338,seeking_diff_profession_str,object,Job Search,,,,,,:ref:`🔗 <w5e-NamesDifferentJobs>`
1339,change_proff_job,category,Job Search,,,,,,:ref:`🔗 <w5e-SearchingTraining>`


In [24]:
if language_short == "e":
    variable_table.to_csv(
        f"{path}variable_table-waves-{'-'.join(str(i) for i in waves)}-{language_short}.csv",
        sep=",",
        index=False,
    )

## Create index file for questions sorted according to topics

In [17]:
if len(waves) == 1:
    topic_table = create_description_table(
        table, waves, language_short, return_links_as_symbols=False, routing=False
    )
    topic_dict = topic_table.groupby("Topic")[f"Links Wave {waves[0]}"].unique()
    for key in ["skip", "Skip"]:
        if key in topic_dict.keys():
            topic_dict = topic_dict.drop(key)
    create_overview_page(topic_dict, waves[0], language_long, path)